Naturally after reading a particular text, Humans can recognize some common entities like person name , date and so on. But to do the same with the aid of computers, we have to help the computer learn and do the task for us. To do so, we can avail services of `Natural Language Processing (NLP)` and `Machine Learning (ML)`. The role of NLP is to make possible for the computer to read text, communicate with humans , understand their sentiments and interpret it by knowing the patterns and rules of languages. And the role of ML is to help machines learn and improve in time.

Like how we define a heartbeat as a two-part pumping action, we define the working of NER as a two-step process

1. Identify the named entity 
2. Categorize the named entity.

### Importing necessary libraries

In [1]:
from __future__ import unicode_literals, print_function

import random
from pathlib import Path
import spacy
from tqdm import tqdm
from spacy.training.example import Example
import pickle

### Creating Blank Model

The very first step in building a cutom model is to create a blank ‘en’ model. This blank model is built to carry out NER process.

In [2]:
model = None
output_dir=Path("ner/")
n_iter=100

#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

Created blank 'en' model


### Training Data

First we need to create entity categories such as `Degree`, `School name`, `Location`, `Percentage` & `Date` and feed the NER model with relevant training data.

Spacy library accepts the training data in the form of tuples containing text data and a dictionary. The dictionary should contain the start and end indices of the named entity in the text and category of the named entity.

In [3]:
TRAIN_DATA = [('Higher School Certificate, Parramatta Marist High School, Westmead (1998)',
                {'entities':[(0,25,'degree'),(27,56,'school_name'),(58,66,'location'),(68,72,'date')]}),
            
            ('Bachelor of Business, University of Western Sydney (2005) ',
            {'entities':[(0,20,'degree'),(22,43,'school_name'),(44,50,'location'),(52,56,'date')]}),
            
            ('2007–2010 BCA (Bachelor of Computer Application) from Khalsa college for women, Amritsar (Affiliated to Guru Nanak Dev University (G.N.D.U) India ',
            {'entities':[(0,9,'date'),(12,50,'degree'),(54,78,'school_name'),(80,88,'location')]}),
            
            ('2010–2013 MCA (Masters in Computer Applications) from Amritsar College of Engineering, Amritsar (Affiliated to Punjab Technical University (P.T.U) India.',
            {'entities':[(0,9,'date'),(10,48,'degree'),(54,85,'school_name'),(87,95,'location')]})]

### Pipeline Set-up

Next step is to set-up the pipeline with only NER using `create_pipe` function.

In [5]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe('ner',last=True)
else:
    ner = nlp.get_pipe

### Training the model

Before starting to train the model, we have to add the categories of the named entities (Labels) to the ‘ner’ using `ner.add_label()` method and then we have to disable other pipeline components apart from ‘ner’ since these components should not get affected while training. we train the recognizer by disabling those components using `nlp.disable_pipes()` method.

To train the ‘ner’ model, the model has to be looped over the training data for sufficient number of iterations. For that, we use `n_iter` which is set to 100. Inorder to ensure that the model does not make generalizations based on the order of the examples, we will shuffle the training data randomly before every iteration using `random.shuffle()` function.

We use `tqdm()` function for creating Progress Meters or Progress Bars. Example class holds the information for one training instance. It stores two objects, one for holding the predictions of the pipeline and other for holding the reference data. `Example.from_dict(doc,annotations)` method is used to construct an Example object from the predicted document (doc) and the reference annotations provided as a dictionary (annotations).The `nlp_update()` function can be used to train the recognizer.

In [ ]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])
        ## check if the labels are added  `ner.labels`

example = []
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example], 
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

### Saving the model

Save the model which is stored in the `output_dir` variable and export the model as a pkl file.

In [15]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)
pickle.dump(nlp, open( "education nlp.pkl", "wb" ))

Saved model to ner


### Testing the trained model

In [18]:
doc=nlp("2015-2017, BE Chemical Engineering, Coimbatore Institute of Technology , India")
for ent in doc.ents:
    print(ent.label_+ '  ------>   ' + ent.text)

date  ------>   2015
date  ------>   2017
school_name  ------>   BE Chemical Engineering
school_name  ------>   Coimbatore Institute of Technology
location  ------>   India


In [19]:
from spacy import displacy

displacy.render(doc, style='ent', jupyter=True)

We can see that `BE Chemical Engineering` is being categorised as `school_name`. Hence we assign entities manually in case they have not been recognised.

`spacky.tokens.Span`  --> [: :] works normally like list comprehension in python.



In [23]:
from spacy.tokens import Span

s1 = Span(doc,3,7,label = 'degree')

doc.set_ents([s1],default='unmodified')

In [27]:
for ent in doc.ents:
    print(ent.label_+ '  ------>   ' + ent.text)

date  ------>   2015
date  ------>   2017
degree  ------>   , BE Chemical Engineering
school_name  ------>   Coimbatore Institute of Technology
location  ------>   India
